In [3]:
import json
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.keyedvectors import KeyedVectors
import os



In [7]:
def dump_pkl(vocab, pkl_path, overwrite=True):
    """
    存储文件
    :param pkl_path:
    :param overwrite:
    :return:
    """
    if pkl_path and os.path.exists(pkl_path) and not overwrite:
        return
    if pkl_path:
        with open(pkl_path, 'wb') as f:
            pickle.dump(vocab, f, protocol=pickle.HIGHEST_PROTOCOL)
            # pickle.dump(vocab, f, protocol=0)
        print("save %s ok." % pkl_path)

def remove_punc(s):
    return s
    #return s.translate(str.maketrans('', '', string.punctuation))

def extract_sentences_squad(squad_file):
    squad_data = json.load(squad_file)
    context_sentences = []
    for d in squad_data['data']:
        for p in d['paragraphs']:
            context = p['context']
            context_sentences.append(remove_punc(context).split(' '))
    return context_sentences       

def extract_sentences_dureader(du_file):
    document_sentences = []
    for line in du_file:
        line_data = json.loads(line)
        for document in line_data['documents']:
            for paragraph in document['segmented_paragraphs']:
                document_sentences.append(paragraph)
    return document_sentences

def build_w2v(sentences, w2v_bin_path):  # one sentence is a list of tokens
    print('train w2v model...')
    w2v = Word2Vec([sentence for sentence in sentences], size=256, window=5, sg=1)
    w2v.wv.save_word2vec_format(w2v_bin_path, binary=True)
    print("save %s ok." % w2v_bin_path)
    #sim = w2v.wv.similarity('技师', '车主')
   # print(sim)
    # load model
#     model = KeyedVectors.load_word2vec_format(w2v_bin_path, binary=True)
#     word_dict = {}
#     for word in model.vocab:
#         word_dict[word] = model[word]
#     dump_pkl(word_dict, out_path, overwrite=True)

In [8]:
squad_file = open('./SQuaAD2.0/dev-v2.0.json')
du_file = open('./dureader-processed/devset/search.dev.json')

squad_sentences = extract_sentences_squad(squad_file)
du_sentences = extract_sentences_dureader(du_file)
build_w2v(squad_sentences, 'squad_w2v.bin')
build_w2v(du_sentences, 'du_w2v.bin')


train w2v model...
save squad_w2v.bin ok.
train w2v model...
save du_w2v.bin ok.
